In [2]:
# Установка необходимых библиотек для Google Colab
print("Установка зависимостей...")
!pip install -q requests beautifulsoup4 lxml pandas nest_asyncio python-dotenv

import os
import sys
import asyncio
import nest_asyncio
import random
from datetime import datetime
from typing import List, Dict, Optional
from dataclasses import dataclass
from IPython.display import display, Markdown, HTML

# Применяем патч для работы asyncio в Colab
nest_asyncio.apply()

# Проверяем и импортируем необходимые библиотеки
try:
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from dotenv import load_dotenv
    load_dotenv()
    print("Все библиотеки успешно импортированы")
except ImportError as e:
    print(f"Ошибка импорта: {e}")
    sys.exit(1)

@dataclass
class Product:
    """Класс для хранения информации о товаре"""
    name: str
    price: float
    url: str
    brand: Optional[str] = None
    category: Optional[str] = None
    rating: Optional[float] = None
    seller: Optional[str] = None
    timestamp: datetime = None

    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now()

class MarketParser:
    """Базовый класс для парсинга маркетплейсов"""

    def __init__(self, base_url: str, headers: Dict = None):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update(headers or {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive'
        })

    def normalize_price(self, price_text: str) -> float:
        """Нормализация цены из текста в число"""
        if not price_text:
            return 0.0

        import re
        numbers = re.findall(r'[\d\s]+', price_text)
        if numbers:
            try:
                price_str = numbers[0].replace(' ', '').replace(',', '.')
                return float(price_str)
            except:
                return 0.0
        return 0.0

    async def search_products(self, query: str, limit: int = 5) -> List[Product]:
        """Асинхронный поиск товаров (базовая реализация)"""
        raise NotImplementedError("Метод должен быть реализован в дочернем классе")

class YandexMarketParser(MarketParser):
    """Парсер для Яндекс.Маркета"""

    def __init__(self):
        super().__init__(
            base_url="https://market.yandex.ru",
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                'Referer': 'https://market.yandex.ru/'
            }
        )

    async def search_products(self, query: str, limit: int = 5) -> List[Product]:
        """Поиск товаров на Яндекс.Маркете"""
        print(f"Поиск '{query}' на Яндекс.Маркете...")

        search_url = f"{self.base_url}/search?text={requests.utils.quote(query)}"

        try:
            response = self.session.get(search_url, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            products = []

            product_cards = soup.find_all('div', {'data-zone-name': 'snippet-card'})

            if not product_cards:
                product_cards = soup.find_all('article', {'data-autotest-id': 'product-snippet'})

            for i, card in enumerate(product_cards[:limit]):
                try:
                    name_elem = card.find(['h3', 'a'], {'class': re.compile(r'title|name|link')})
                    name = name_elem.get_text(strip=True) if name_elem else f"Товар {i+1}"

                    price_elem = card.find(['span', 'div'], {'class': re.compile(r'price|value|currency')})
                    price_text = price_elem.get_text(strip=True) if price_elem else "0"
                    price = self.normalize_price(price_text)

                    link_elem = card.find('a', href=True)
                    url = link_elem['href'] if link_elem else search_url
                    if url and not url.startswith('http'):
                        url = self.base_url + url

                    brand_elem = card.find(['span', 'div'], {'class': re.compile(r'brand|company')})
                    brand = brand_elem.get_text(strip=True) if brand_elem else None

                    rating_elem = card.find(['span', 'div'], {'class': re.compile(r'rating|stars')})
                    rating = None
                    if rating_elem:
                        rating_text = rating_elem.get_text(strip=True)
                        rating_match = re.search(r'[\d\.]+', rating_text)
                        if rating_match:
                            rating = float(rating_match.group())

                    product = Product(
                        name=name[:100],
                        price=price,
                        url=url,
                        brand=brand,
                        rating=rating,
                        seller="Яндекс.Маркет"
                    )

                    products.append(product)

                except Exception as e:
                    print(f"Ошибка обработки товара {i+1}: {str(e)[:50]}")
                    continue

            if not products:
                print("Использую демо-данные для Яндекс.Маркета")
                products = self._get_demo_products(query, limit)

            return products

        except Exception as e:
            print(f"Ошибка поиска: {e}")
            return self._get_demo_products(query, limit)

    def _get_demo_products(self, query: str, limit: int) -> List[Product]:
        """Демонстрационные данные для Яндекс.Маркета"""
        demo_products = [
            Product(
                name=f"Смартфон {query} X200 Pro",
                price=34999.0,
                url="https://market.yandex.ru/product--smartfon-x200-pro/123456",
                brand="Xiaomi",
                rating=4.5,
                seller="DNS"
            ),
            Product(
                name=f"Ноутбук {query} Gaming 15",
                price=89999.0,
                url="https://market.yandex.ru/product--noutbuk-gaming-15/234567",
                brand="ASUS",
                rating=4.8,
                seller="М.Видео"
            ),
            Product(
                name=f"Наушники {query} Wireless Pro",
                price=12999.0,
                url="https://market.yandex.ru/product--naushniki-wireless-pro/345678",
                brand="Sony",
                rating=4.7,
                seller="Ситилинк"
            ),
            Product(
                name=f"Планшет {query} Tab S9",
                price=65999.0,
                url="https://market.yandex.ru/product--planshet-tab-s9/456789",
                brand="Samsung",
                rating=4.6,
                seller="Эльдорадо"
            ),
            Product(
                name=f"Часы {query} Watch 5",
                price=23999.0,
                url="https://market.yandex.ru/product--chasy-watch-5/567890",
                brand="Apple",
                rating=4.9,
                seller="re:Store"
            )
        ]
        return demo_products[:limit]

class AvitoParser(MarketParser):
    """Парсер для Авито"""

    def __init__(self):
        super().__init__(
            base_url="https://www.avito.ru",
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                'Referer': 'https://www.avito.ru/'
            }
        )

    async def search_products(self, query: str, limit: int = 5) -> List[Product]:
        """Поиск товаров на Авито"""
        print(f"Поиск '{query}' на Авито...")

        search_url = f"{self.base_url}/all?q={requests.utils.quote(query)}"

        try:
            response = self.session.get(search_url, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            products = []

            product_cards = soup.find_all('div', {'data-marker': 'item'})

            if not product_cards:
                product_cards = soup.find_all('div', {'class': re.compile(r'iva-item-root')})

            for i, card in enumerate(product_cards[:limit]):
                try:
                    name_elem = card.find(['h3', 'a'], {'class': re.compile(r'title|link')})
                    name = name_elem.get_text(strip=True) if name_elem else f"Объявление {i+1}"

                    price_elem = card.find(['span', 'meta'], {'itemprop': 'price'})
                    if not price_elem:
                        price_elem = card.find(['span'], {'class': re.compile(r'price-text')})
                    price_text = price_elem.get_text(strip=True) if price_elem else "0"
                    price = self.normalize_price(price_text)

                    link_elem = card.find('a', href=True)
                    url = link_elem['href'] if link_elem else search_url
                    if url and not url.startswith('http'):
                        url = self.base_url + url

                    location_elem = card.find(['div', 'span'], {'class': re.compile(r'geo|location')})
                    location = location_elem.get_text(strip=True) if location_elem else "Не указано"

                    product = Product(
                        name=name[:100],
                        price=price,
                        url=url,
                        seller=location,
                        category="Частное объявление"
                    )

                    products.append(product)

                except Exception as e:
                    print(f"Ошибка обработки объявления {i+1}: {str(e)[:50]}")
                    continue

            if not products:
                print("Использую демо-данные для Авито")
                products = self._get_demo_products(query, limit)

            return products

        except Exception as e:
            print(f"Ошибка поиска: {e}")
            return self._get_demo_products(query, limit)

    def _get_demo_products(self, query: str, limit: int) -> List[Product]:
        """Демонстрационные данные для Авито"""
        demo_products = [
            Product(
                name=f"{query} б/у в хорошем состоянии",
                price=15000.0,
                url="https://www.avito.ru/moskva/telefony/iphone_12_256gb_123456",
                seller="Москва, ЦАО",
                category="Электроника"
            ),
            Product(
                name=f"{query} новый с гарантией",
                price=25000.0,
                url="https://www.avito.ru/sankt-peterburg/elektronika/noutbuk_hp_234567",
                seller="Санкт-Петербург",
                category="Компьютеры"
            ),
            Product(
                name=f"{query} от собственника",
                price=45000.0,
                url="https://www.avito.ru/ekaterinburg/avtotovary/shiny_зимние_345678",
                seller="Екатеринбург",
                category="Автотовары"
            ),
            Product(
                name=f"{query} недорого",
                price=8000.0,
                url="https://www.avito.ru/kazan/odezhda_obuv_aksessuary/kurtka_зимняя_456789",
                seller="Казань",
                category="Одежда"
            ),
            Product(
                name=f"{query} срочная продажа",
                price=12000.0,
                url="https://www.avito.ru/novosibirsk/bytovaya_elektronika/televizor_samsung_567890",
                seller="Новосибирск",
                category="Бытовая техника"
            )
        ]
        return demo_products[:limit]

class ResultFormatter:
    """Класс для форматирования результатов в Markdown"""

    @staticmethod
    def format_to_markdown(products: List[Product], marketplace: str, query: str) -> str:
        """Форматирование списка товаров в Markdown таблицу"""

        markdown = f"## Результаты поиска: '{query}'\n"
        markdown += f"**Платформа:** {marketplace}\n"
        markdown += f"**Найдено товаров:** {len(products)}\n"
        markdown += f"**Дата поиска:** {datetime.now().strftime('%d.%m.%Y %H:%M')}\n\n"

        markdown += "| № | Название | Цена (₽) | Продавец/Бренд | Ссылка |\n"
        markdown += "|---|----------|----------|----------------|--------|\n"

        for i, product in enumerate(products, 1):
            name_display = product.name
            if len(name_display) > 40:
                name_display = name_display[:37] + "..."

            seller_display = product.brand or product.seller or "Не указано"
            if len(seller_display) > 20:
                seller_display = seller_display[:17] + "..."

            price_display = f"{product.price:,.0f}".replace(',', ' ') if product.price > 0 else "—"

            link_display = f"[Открыть]({product.url})" if product.url else "—"

            markdown += f"| {i} | {name_display} | {price_display} ₽ | {seller_display} | {link_display} |\n"

        valid_prices = [p.price for p in products if p.price > 0]
        if valid_prices:
            min_price = min(valid_prices)
            max_price = max(valid_prices)
            avg_price = sum(valid_prices) / len(valid_prices)

            markdown += f"\n**Статистика цен:**\n"
            markdown += f"- Минимальная: **{min_price:,.0f} ₽**\n"
            markdown += f"- Максимальная: **{max_price:,.0f} ₽**\n"
            markdown += f"- Средняя: **{avg_price:,.0f} ₽**\n"

        return markdown

    @staticmethod
    def format_to_dataframe(products: List[Product]) -> pd.DataFrame:
        """Форматирование в DataFrame для анализа"""
        data = []
        for product in products:
            data.append({
                'Название': product.name,
                'Цена (₽)': product.price,
                'Бренд': product.brand or '—',
                'Продавец': product.seller or '—',
                'Рейтинг': product.rating or '—',
                'Ссылка': product.url,
                'Дата': product.timestamp.strftime('%d.%m.%Y %H:%M')
            })

        return pd.DataFrame(data)

async def search_on_marketplace(parser: MarketParser, query: str, limit: int = 5) -> Dict:
    """Выполняет поиск на маркетплейсе и возвращает результаты"""

    print(f"\n{'='*60}")
    print(f"Запуск поиска на {parser.__class__.__name__}")
    print(f"Запрос: '{query}'")
    print(f"Лимит: {limit} товаров")
    print('='*60)

    start_time = datetime.now()
    products = await parser.search_products(query, limit)
    search_time = (datetime.now() - start_time).total_seconds()

    print(f"Найдено товаров: {len(products)}")
    print(f"Время поиска: {search_time:.2f} секунд")

    return {
        'marketplace': parser.__class__.__name__.replace('Parser', ''),
        'products': products,
        'query': query,
        'search_time': search_time
    }

def display_results(results_list: List[Dict]):
    """Отображает результаты из всех маркетплейсов"""

    print("\n" + "="*60)
    print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ ПОИСКА")
    print("="*60)

    for result in results_list:
        marketplace = result['marketplace']
        products = result['products']
        query = result['query']

        markdown_content = ResultFormatter.format_to_markdown(products, marketplace, query)

        display(Markdown(markdown_content))

        df = ResultFormatter.format_to_dataframe(products)
        print(f"\nТаблица данных ({marketplace}):")
        print(df[['Название', 'Цена (₽)', 'Бренд', 'Продавец']].to_string(index=False))

        print("\n" + "-"*60)

async def main():
    """Основная функция программы"""

    print("СИСТЕМА ПОИСКА ТОВАРОВ НА МАРКЕТПЛЕЙСАХ")
    print("="*60)

    print("\nДоступные маркетплейсы:")
    print("1. Яндекс.Маркет (электроника, техника)")
    print("2. Авито (объявления, б/у товары)")
    print("3. Все платформы")

    try:
        choice = input("\nВыберите маркетплейс (1-3): ").strip()

        query = input("Введите поисковый запрос: ").strip()
        if not query:
            query = "смартфон"

        limit_input = input("Сколько товаров искать? (по умолчанию 5): ").strip()
        limit = int(limit_input) if limit_input.isdigit() and int(limit_input) > 0 else 5

        results = []

        if choice == "1":
            parser = YandexMarketParser()
            result = await search_on_marketplace(parser, query, limit)
            results.append(result)

        elif choice == "2":
            parser = AvitoParser()
            result = await search_on_marketplace(parser, query, limit)
            results.append(result)

        elif choice == "3":
            parsers = [YandexMarketParser(), AvitoParser()]
            tasks = [search_on_marketplace(parser, query, limit) for parser in parsers]
            results = await asyncio.gather(*tasks)

        else:
            print("Неверный выбор. Использую Яндекс.Маркет по умолчанию.")
            parser = YandexMarketParser()
            result = await search_on_marketplace(parser, query, limit)
            results.append(result)

        display_results(results)

        save_results(results)

    except KeyboardInterrupt:
        print("\n\nПоиск прерван пользователем")
    except Exception as e:
        print(f"\nКритическая ошибка: {e}")
        import traceback
        traceback.print_exc()

def save_results(results_list: List[Dict]):
    """Сохраняет результаты в файлы"""

    print("\nСохранение результатов...")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for result in results_list:
        marketplace = result['marketplace']
        products = result['products']
        query = result['query']

        markdown_content = ResultFormatter.format_to_markdown(products, marketplace, query)
        filename_md = f"{marketplace}_{query}_{timestamp}.md"

        with open(filename_md, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"Markdown сохранен: {filename_md}")

        df = ResultFormatter.format_to_dataframe(products)
        filename_csv = f"{marketplace}_{query}_{timestamp}.csv"
        df.to_csv(filename_csv, index=False, encoding='utf-8-sig')
        print(f"CSV сохранен: {filename_csv}")

        import json
        products_data = []
        for product in products:
            products_data.append({
                'name': product.name,
                'price': product.price,
                'url': product.url,
                'brand': product.brand,
                'seller': product.seller,
                'rating': product.rating,
                'timestamp': product.timestamp.isoformat()
            })

        filename_json = f"{marketplace}_{query}_{timestamp}.json"
        with open(filename_json, 'w', encoding='utf-8') as f:
            json.dump({
                'marketplace': marketplace,
                'query': query,
                'timestamp': timestamp,
                'products': products_data
            }, f, ensure_ascii=False, indent=2, default=str)
        print(f"JSON сохранен: {filename_json}")

    print("\nВсе файлы сохранены в текущей директории")

def run_search_system():
    """Запуск системы поиска товаров в Google Colab"""

    print("Запуск системы поиска товаров...")
    print("\n" + "="*60)
    print("ИНФОРМАЦИЯ:")
    print("- Система ищет товары на российских маркетплейсах")
    print("- Используются демо-данные при недоступности сайтов")
    print("- Результаты сохраняются в 3 форматах: MD, CSV, JSON")
    print("="*60)

    try:
        asyncio.run(main())
    except RuntimeError as e:
        if "event loop" in str(e):
            print("Обнаружен запущенный event loop, создаю новый...")
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            loop.run_until_complete(main())
        else:
            raise

    print("\n" + "="*60)
    print("СИСТЕМА ЗАВЕРШИЛА РАБОТУ")
    print("="*60)

def run_demo_mode():
    """Запуск демонстрационного режима без поиска"""

    print("ДЕМОНСТРАЦИОННЫЙ РЕЖИМ")
    print("="*60)

    demo_products = [
        Product(
            name="Смартфон Samsung Galaxy S23 Ultra",
            price=89999.0,
            url="https://market.yandex.ru/product--samsung-galaxy-s23-ultra/123",
            brand="Samsung",
            rating=4.8,
            seller="М.Видео"
        ),
        Product(
            name="Ноутбук Apple MacBook Pro 16",
            price=199999.0,
            url="https://market.yandex.ru/product--macbook-pro-16/456",
            brand="Apple",
            rating=4.9,
            seller="re:Store"
        ),
        Product(
            name="Наушники Sony WH-1000XM5",
            price=29999.0,
            url="https://market.yandex.ru/product--sony-wh-1000xm5/789",
            brand="Sony",
            rating=4.7,
            seller="DNS"
        )
    ]

    markdown = ResultFormatter.format_to_markdown(demo_products, "Яндекс.Маркет (демо)", "премиум техника")
    display(Markdown(markdown))

    print("\n" + "="*60)
    print("Это демонстрационные данные")
    print("Для реального поиска запустите функцию run_search_system()")
    print("="*60)

import re

if __name__ == "__main__":
    print("СИСТЕМА ПОИСКА ТОВАРОВ V1.0")
    print("\nВыберите режим:")
    print("1. Полный поиск на маркетплейсах")
    print("2. Демонстрационный режим (без реального поиска)")

    try:
        choice = input("\nВведите номер режима (1 или 2): ").strip()

        if choice == "1":
            run_search_system()
        elif choice == "2":
            run_demo_mode()
        else:
            print("Неверный выбор. Запускаю демо-режим...")
            run_demo_mode()

    except KeyboardInterrupt:
        print("\n\nПрограмма завершена пользователем")
    except Exception as e:
        print(f"\nОшибка: {e}")
        print("Запускаю демонстрационный режим...")
        run_demo_mode()

Установка зависимостей...
Все библиотеки успешно импортированы
СИСТЕМА ПОИСКА ТОВАРОВ V1.0

Выберите режим:
1. Полный поиск на маркетплейсах
2. Демонстрационный режим (без реального поиска)

Введите номер режима (1 или 2): 2
ДЕМОНСТРАЦИОННЫЙ РЕЖИМ


## Результаты поиска: 'премиум техника'
**Платформа:** Яндекс.Маркет (демо)
**Найдено товаров:** 3
**Дата поиска:** 12.01.2026 10:51

| № | Название | Цена (₽) | Продавец/Бренд | Ссылка |
|---|----------|----------|----------------|--------|
| 1 | Смартфон Samsung Galaxy S23 Ultra | 89 999 ₽ | Samsung | [Открыть](https://market.yandex.ru/product--samsung-galaxy-s23-ultra/123) |
| 2 | Ноутбук Apple MacBook Pro 16 | 199 999 ₽ | Apple | [Открыть](https://market.yandex.ru/product--macbook-pro-16/456) |
| 3 | Наушники Sony WH-1000XM5 | 29 999 ₽ | Sony | [Открыть](https://market.yandex.ru/product--sony-wh-1000xm5/789) |

**Статистика цен:**
- Минимальная: **29,999 ₽**
- Максимальная: **199,999 ₽**
- Средняя: **106,666 ₽**



Это демонстрационные данные
Для реального поиска запустите функцию run_search_system()
